In [21]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def get_realtime_price(url: str = 'https://finance.naver.com/sise/sise_index.naver?code=KOSDAQ') :
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    cur_price = dom.select_one('em#now_value').text
    cur_volume = dom.select_one('td#quant').text
    cur_volume_price = dom.select_one('td#amount').text
    cur_volume_personal = dom.select_one('dl.lst_kos_info').select_one(':nth-child(2)').select_one('span').text
    cur_volume_forg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(3)').select_one('span').text
    cur_volume_instfund = dom.select_one('dl.lst_kos_info').select_one(':nth-child(4)').select_one('span').text

    cur_volume_prog_pos = dom.select_one('dl.lst_kos_info').select_one(':nth-child(6)').select_one('span').text
    cur_volume_prog_neg = dom.select_one('dl.lst_kos_info').select_one(':nth-child(7)').select_one('span').text
    
    print(cur_volume_personal + '|' + cur_volume_forg + '|' + cur_volume_instfund + '|' 
          + cur_volume_prog_pos + '|' + cur_volume_prog_neg)
    print(cur_price + "|" + cur_volume + "|" + cur_volume_price + "|" )
    

get_realtime_price()
    

+813억|-804억|-25억|+7억|-793억
784.37|394,013|2,761,851|
